# Generate object(sign) and annotations on draw

In [158]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
from glob import glob
from matplotlib import pyplot
from PIL import Image
from PIL import ImageFilter

import numpy as np

from lxml import etree
import xml.etree.ElementTree as xml
import random
import cv2

In [2]:
draw_data = glob('./data/backGround/draw/*.png')
annotaions_data = glob('./data/backGround/annotations/*.xml')
sign_data = glob('./data/backGround/sign/*.png')

# Reshape Sign 
size = 50, 50

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im.save(s)

# Change color of sign

In [3]:
def rgb_to_hsv(r, g, b):
    
    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = (df/mx)*100
    v = mx*100
    
    return h, s, v

In [96]:
def ChangeColor(inputImg,r2,g2,b2):
    
    im = cv2.imread(inputImg)
    data = np.array(im)
    
    imName = inputImg.split('sign/')[1]
    imName = imName.split('.png')[0]    

    r1, g1, b1 = 0, 0, 0 # Original value
    r2, g2, b2 = r2,g2,b2 # Value that we want to replace it with

    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:,:,:3][mask] = [r2, g2, b2]

    im = Image.fromarray(data)    
    imDir = './data/backGround/sign/COR_'+imName+'.png'
    im.save(imDir)
    
    return imDir

In [181]:
def ChangeColor2(inputImg,r2,g2,b2):
    
    im = cv2.imread(inputImg)
    
    kernel_size_row = 3
    kernel_size_col = 3
    kernel = np.ones((3, 3), np.uint8)          
    
    im = cv2.dilate(im, kernel, iterations=1)
    
    ret, th1 = cv2.threshold(im, 247, 255, cv2.THRESH_BINARY)
   
    imName = inputImg.split('sign/')[1]
    imName = imName.split('.png')[0]    
    
    data = np.array(th1)   
            
    r1, g1, b1 = 0, 0, 0 # Original value
    r2, g2, b2 = r2,g2,b2 # Value that we want to replace it with
    
    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:,:,:3][mask] = [r2, g2, b2]
    
    
    im = Image.fromarray(data)    
    imDir = './data/backGround/sign/COR_'+imName+'.png'
    im.save(imDir)
       
    return imDir

# Make n-Sign on Draw with loaction(x,y) information 

In [182]:
def genData(clss, amount, dr_fileName,r2,g2,b2):
    
    l_img =  cv2.imread(dr_fileName)
    s_img =  cv2.imread('./data/backGround/sign/'+clss+'.png',-1)  
    
    signDir = './data/backGround/sign/'+clss+'.png'
    
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s
    
    dr_fileName = dr_fileName.split('draw/')[1]
    dr_fileName = dr_fileName.split('.png')[0]    
    
    new_dirFileName = './data/draw/dataset/GN_'+dr_fileName+str(random.random())+'.png'          
    new_fileName  = new_dirFileName .split('dataset/')[1]

    imName = ChangeColor2(signDir,r2,g2,b2)
    s_img =  cv2.imread(imName,-1)
          
    h,w,d = l_img.shape
      
    # make annotations file
    root = etree.Element("annotation")

    folder = etree.SubElement(root, "folder").text ='draw'  
    filename =  etree.SubElement(root, "filename").text = new_fileName    
    path =  etree.SubElement(root, "path").text = new_dirFileName

    source =  etree.SubElement(root, "source")
    database =  etree.SubElement(source, "database").text= 'Unknown'     
    
    size =  etree.SubElement(root, "size")
    width =  etree.SubElement(size, "width").text = str(w)
    height =  etree.SubElement(size, "height").text = str(h)
    depth =  etree.SubElement(size, "depth").text = str(d)

    segmented =  etree.SubElement(root, "segmented").text = '0'
    
       
    # generate signs and annotaion files.                 
    for i in range(amount):    
               
        xmin = random.randrange(100,w)
        ymin = random.randrange(100,h)
                
        #x_offset=y_offset=16
        
        x1, x2 = xmin,  xmin + s_img.shape[1] 
        y1, y2 = ymin,  ymin + s_img.shape[0] 
        
        print(x1, x2, y1, y2)
        
        obj =  etree.SubElement(root, "object")
        name =   etree.SubElement(obj, "name").text = clss
        poset =   etree.SubElement(obj, "pose").text = 'Unspecified'
        truncated =   etree.SubElement(obj, "truncated").text = '0'
        difficult =   etree.SubElement(obj, "difficult").text = '0'
        
        bndbox = etree.SubElement(obj, "bndbox")
        xmin =  etree.SubElement(bndbox, "xmin").text = str(x1)
        ymin =  etree.SubElement(bndbox, "ymin").text = str(y1)
        xmax =  etree.SubElement(bndbox, "xmax").text = str(x2)
        ymax =  etree.SubElement(bndbox, "ymax").text = str(y2)
        
        for c in range(0, 3):
            l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])

    cv2.imwrite(new_fileName, l_img)        
    tree = etree.ElementTree(root)
    
    with open('./data/backGround/'+new_fileName+'.xml', "bw") as fh:
        tree.write(fh)
         
      

In [183]:
genData('DT_150',10, './data/backGround/draw/b1.png',255,255,72)

250 300 280 307
1305 1355 445 472
1163 1213 561 588
1057 1107 207 234
155 205 576 603
1090 1140 413 440
988 1038 591 618
546 596 861 888
274 324 671 698
355 405 528 555
